# learner

In [ ]:
#| default_exp learner

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#|export
import torch, re, math, numpy as np, os
import torch.nn as nn
from typing import Any, Tuple, Optional, Sequence, Union, Dict, List, NamedTuple

from transformers.trainer_pt_utils import find_batch_size, nested_concat, nested_numpify, IterableDatasetShard
from transformers.trainer_utils import has_length, denumpify_detensorize, speed_metrics
from transformers.modeling_utils import unwrap_model
from transformers import AutoTokenizer, BatchEncoding, Seq2SeqTrainer, Seq2SeqTrainingArguments

from torch.utils.data import DataLoader, Dataset

from torch.nn.parallel import DataParallel
from torch.nn.parallel._functions import Scatter
from torch.nn.parallel.scatter_gather import _is_namedtuple

from xcai.data import *
from xcai.generation.trie import Trie
from xcai.generation.generate import *

from fastcore.utils import *
from fastcore.meta import *
from fastcore.dispatch import *

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [ ]:
from xcai.test_utils import *
from xcai.models.BT000X import *
from xcai.metrics import *

## Setup

In [ ]:
block = Test.from_cfg('train')

/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/scipy/sparse/_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
bsz = 10
batch = block.train.one_batch(bsz)

In [ ]:
m = BT0002.from_pretrained('bert-base-cased', tn_targ=10_000, ig_tok=0)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of BT0002 were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['loss_fn.o']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
b = prepare_batch(m, batch)

In [ ]:
m = m.to('cuda')
b = b.to('cuda')

In [ ]:
o = m(**b)

In [ ]:
o.loss

tensor(15.9060, device='cuda:0', grad_fn=<SumBackward0>)

## DataParallel

In [ ]:
#| export
def scatter(inputs, target_gpus, chunk_sizes=None, dim=0):
    def scatter_map(obj):
        if isinstance(obj, torch.Tensor):
            return Scatter.apply(target_gpus, chunk_sizes, dim, obj)
        if _is_namedtuple(obj):
            return [type(obj)(*args) for args in zip(*map(scatter_map, obj))]
        if isinstance(obj, tuple) and len(obj) > 0:
            return list(zip(*map(scatter_map, obj)))
        if isinstance(obj, list) and len(obj) > 0:
            return [list(i) for i in zip(*map(scatter_map, obj))]
        if isinstance(obj, dict) and len(obj) > 0:
            return [type(obj)(i) for i in zip(*map(scatter_map, obj.items()))]
        return [obj for _ in target_gpus] 
    try:
        res = scatter_map(inputs)
    finally:
        scatter_map = None
    return res
    
def scatter_kwargs(
    inputs: Tuple[Any, ...],
    kwargs: Optional[Dict[str, Any]],
    target_gpus: Sequence[Union[int, torch.device]],
    chunk_sizes: Optional[Sequence[int]]=None,
    dim: int = 0,
) -> Tuple[Tuple[Any, ...], Tuple[Dict[str, Any], ...]]:
    scattered_inputs = scatter(inputs, target_gpus, chunk_sizes, dim) if inputs else []
    scattered_kwargs = scatter(kwargs, target_gpus, chunk_sizes, dim) if kwargs else []
    if len(scattered_inputs) < len(scattered_kwargs):
        scattered_inputs.extend(() for _ in range(len(scattered_kwargs) - len(scattered_inputs)))
    elif len(scattered_kwargs) < len(inputs):
        scattered_kwargs.extend({} for _ in range(len(scattered_inputs) - len(scattered_kwargs)))
    return scattered_inputs, scattered_kwargs
    

In [ ]:
#| export
class XCDataParallel(DataParallel):

    @delegates(DataParallel.__init__)
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def _get_meta_name(self, x:Optional[Dict[str, Any]]):
        return list(set([k.split('_')[0] for k in x]).difference(['data', 'lbl2data'])) if len(x) else set()

    def _extract_feat(self, x:Optional[Dict[str, Any]], prefix:str):
        return {k:v for k,v in x.items() if re.match(f'^{prefix}', k) and not re.match(r'.*2ptr$', k)}

    def scatter(
        self,
        inputs: Tuple[Any, ...],
        kwargs: Optional[Dict[str, Any]],
        device_ids: Sequence[Union[int, torch.device]],
    ) -> Any:
        if len(inputs): raise ValueError('`inputs` should be empty.')
        meta_name = self._get_meta_name(kwargs)+['lbl2data']
        
        data_feat = self._extract_feat(kwargs, 'data')
        scattered_inputs, scattered_kwargs = scatter_kwargs(inputs, data_feat, device_ids, None, dim=self.dim)
        
        for o in meta_name:
            pn, chunk_sizes = f'{o}_data2ptr', None
            if pn in kwargs:
                ptr = kwargs[pn]
                psz, csz = ptr.shape[0], math.ceil(ptr.shape[0]/len(device_ids))
                psz = (psz//csz+1)*csz if psz%csz else psz
                chunk_sizes = [ptr[p:q].sum() for p,q in zip(range(0, psz+1, csz), range(csz, psz+1, csz))]
                _, sc_ptr = scatter_kwargs(inputs, {pn:ptr}, device_ids, None, dim=self.dim)
                for p,q in zip(scattered_kwargs, sc_ptr): p.update(q)
                    
            feat = self._extract_feat(kwargs, o)    
            _, sc_kwargs = scatter_kwargs(inputs, feat, device_ids, chunk_sizes, dim=self.dim)
            for p,q in zip(scattered_kwargs, sc_kwargs): p.update(q)
            
        return tuple(scattered_inputs), tuple(scattered_kwargs)
        

### Example

In [ ]:
class MyModel(nn.Module):

    def forward(self, **kwargs):
        for k,v in kwargs.items(): print(k, ': ', v.shape, ', ', v.device)
        return
        

In [ ]:
m = XCDataParallel(module=MyModel())

In [ ]:
o = m(**batch)

data_input_ids :  torch.Size([5, 11]) ,  cuda:0
data_token_type_ids :  torch.Size([5, 11]) ,  cuda:0
data_attention_mask :  torch.Size([5, 11]) ,  cuda:0
lbl2data_data2ptr :  torch.Size([5]) ,  cuda:0
lbl2data_idx :  torch.Size([10]) ,  cuda:0
lbl2data_input_ids :  torch.Size([10, 12]) ,  cuda:0
lbl2data_token_type_ids :  torch.Size([10, 12]) ,  cuda:0
lbl2data_attention_mask :  torch.Size([10, 12]) ,  cuda:0
data_input_ids :  torch.Size([5, 11]) ,  cuda:1
data_token_type_ids :  torch.Size([5, 11]) ,  cuda:1
data_attention_mask :  torch.Size([5, 11]) ,  cuda:1
lbl2data_data2ptr :  torch.Size([5]) ,  cuda:1
lbl2data_idx :  torch.Size([26]) ,  cuda:1
lbl2data_input_ids :  torch.Size([26, 12]) ,  cuda:1
lbl2data_token_type_ids :  torch.Size([26, 12]) ,  cuda:1
lbl2data_attention_mask :  torch.Size([26, 12]) ,  cuda:1


In [ ]:
batch.keys()

dict_keys(['lbl2data_idx', 'lbl2data_input_ids', 'lbl2data_token_type_ids', 'lbl2data_attention_mask', 'lbl2data_data2ptr', 'data_input_ids', 'data_token_type_ids', 'data_attention_mask'])

## XCTrie

In [ ]:
#| export
class XCTrie:
    
    @classmethod
    def from_block(cls, block:XCDataBlock, meta:Optional[List]=None):
        lbl_toks = block.lbl_info['input_ids']
        lbl_info = [[i] for i in range(len(lbl_toks))]
        
        trie = Trie.from_list(lbl_toks, lbl_info)

        if meta is not None:
            meta_dset = block.train.dset.meta
            for o in meta:
                if f'{o}_meta' not in meta_dset: raise ValueError(f'`{o}_meta` does not exist.')
                meta_toks = meta_dset[f'{o}_meta'].meta_info['input_ids']
                lbl_meta = meta_dset[f'{o}_meta'].lbl_meta.T.tocsr()
                meta_info = [o.indices.tolist() for o in lbl_meta]
                if len(meta_toks) != len(meta_info): raise ValueError(f'`meta_toks` and `meta_info` should have equal length.')
                trie.update(meta_toks, meta_info)
                
        return trie
        

## Learner

In [ ]:
#| export
class XCEvalLoopOutput(NamedTuple):
    pred_idx: Union[np.ndarray, Tuple[np.ndarray]]
    pred_ptr: Union[np.ndarray, Tuple[np.ndarray]]
    pred_score: Union[np.ndarray, Tuple[np.ndarray]]
    targ_idx: Optional[Union[np.ndarray, Tuple[np.ndarray]]]
    targ_ptr: Optional[Union[np.ndarray, Tuple[np.ndarray]]]
    metrics: Optional[Dict[str, float]]
    num_samples: Optional[int]

class XCPredictionOutput(NamedTuple):
    pred_idx: Union[np.ndarray, Tuple[np.ndarray]]
    pred_ptr: Union[np.ndarray, Tuple[np.ndarray]]
    pred_score: Optional[Union[np.ndarray, Tuple[np.ndarray]]]
    metrics: Optional[Dict[str, float]]
    num_samples: Optional[int]
    

In [ ]:
#| export
class XCLearningArguments(Seq2SeqTrainingArguments):

    @delegates(Seq2SeqTrainingArguments.__init__)
    def __init__(self, 
                 generation_length_penalty:Optional[float]=1.0, 
                 generation_num_beams:Optional[int]=5, 
                 **kwargs):
        super().__init__(**kwargs)
        self.generation_num_beams = generation_num_beams
        self.generation_length_penalty = generation_length_penalty
    

In [ ]:
#| export
class XCLearner(Seq2SeqTrainer):

    @delegates(Seq2SeqTrainer.__init__)
    def __init__(self, 
                 trie:Trie, 
                 **kwargs):
        super().__init__(**kwargs)
        self.tbs = TrieBeamSearch(trie, n_bm=self.args.generation_num_beams, len_penalty=self.args.generation_length_penalty)

    def _wrap_model(self, model, training=True, dataloader=None):
        if unwrap_model(model) is not model:
            return model

        if self.args.n_gpu > 1:
            model = XCDataParallel(module=model)
        return model

    def evaluate(self:XCLearner, eval_dataset:Optional[Dataset]=None, ignore_keys:Optional[List[str]]=None, metric_key_prefix:str="eval",
                 **gen_kwargs):
        gen_kwargs = gen_kwargs.copy()
        if gen_kwargs.get("length_penalty") is None and self.args.generation_length_penalty is not None:
            gen_kwargs["length_penalty"] = self.args.generation_length_penalty
        self.gather_function, self._gen_kwargs  = self.accelerator.gather, gen_kwargs
        return super().evaluate(eval_dataset, ignore_keys=ignore_keys, metric_key_prefix=metric_key_prefix)
    
    def predict(self:XCLearner, test_dataset: Dataset, ignore_keys:Optional[List[str]]=None, metric_key_prefix: str = "test",**gen_kwargs):
        gen_kwargs = gen_kwargs.copy()
        if gen_kwargs.get("length_penalty") is None and self.args.generation_length_penalty is not None:
            gen_kwargs["length_penalty"] = self.args.generation_length_penalty
        self.gather_function, self._gen_kwargs = self.accelerator.gather, gen_kwargs
        self._memory_tracker.start()
        
        test_dataloader = self.get_test_dataloader(test_dataset)
        start_time = time.time()

        eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else self.evaluation_loop
        output = eval_loop(test_dataloader, description="Prediction", ignore_keys=ignore_keys, metric_key_prefix=metric_key_prefix)
        total_batch_size = self.args.eval_batch_size * self.args.world_size
        if f"{metric_key_prefix}_jit_compilation_time" in output.metrics:
            start_time += output.metrics[f"{metric_key_prefix}_jit_compilation_time"]
        output.metrics.update(
            speed_metrics(metric_key_prefix,start_time,num_samples=output.num_samples,num_steps=math.ceil(output.num_samples / total_batch_size),)
        )
        self.control = self.callback_handler.on_predict(self.args, self.state, self.control, output.metrics)
        self._memory_tracker.stop_and_update_metrics(output.metrics)
        return XCPredictionOutput(pred_idx=output.pred_idx, pred_ptr=output.pred_ptr, pred_score=output.pred_score, metrics=output.metrics, num_samples=output.num_samples)
    
    def _gather_host_output(self, output, host_output):
        if output is not None:
            output = self.accelerator.pad_across_processes(output, dim=1, pad_index=-100)
            output = self.gather_function((output))
            return output if host_output is None else nested_concat(host_output, output, padding_index=-100)
        else: return host_output

    def _gather_all_output(self, host_output, all_output):
        if host_output is not None:
            if isinstance(host_output, torch.Tensor): host_output = host_output.cpu()
            return host_output if all_output is None else nested_concat(all_output, host_output, padding_index=-100)
        else: return all_output
            

In [ ]:
#| export
@patch
def prediction_step(
    self:XCLearner,
    model: nn.Module,
    inputs: Dict[str, Union[torch.Tensor, Any]],
    prediction_loss_only: bool,
    ignore_keys: Optional[List[str]] = None,
    **gen_kwargs,
) -> Tuple[Optional[float], Optional[torch.Tensor], Optional[torch.Tensor]]:
    
    with torch.no_grad():
        with self.compute_loss_context_manager(): outputs = model(**inputs)
        loss = (outputs["loss"] if isinstance(outputs, dict) else outputs[0]).mean().detach()
    if self.args.prediction_loss_only or prediction_loss_only: return loss, None
        
    inputs = self._prepare_inputs(inputs)
    n_bm = gen_kwargs.pop("num_beams") if "num_beams" in gen_kwargs and gen_kwargs["num_beams"] is not None else self.args.generation_num_beams
    len_penalty = gen_kwargs.pop("length_penalty") if "length_penalty" in gen_kwargs and gen_kwargs["length_penalty"] is not None else self.args.generation_length_penalty

    generation_inputs = inputs.copy()
    with torch.no_grad(): o = self.tbs.proc(self.model, generation_inputs, n_bm=n_bm, len_penalty=len_penalty)
        
    output = {'pred_idx':o['info2seq2data_idx'], 'pred_score':o['info2seq2data_score'], 'pred_ptr':o['info2seq2data_data2ptr']}
    labels = {'targ_idx':inputs['lbl2data_idx'], 'targ_ptr':inputs['lbl2data_data2ptr']} if 'lbl2data_idx' in inputs else None
    if labels is not None: output.update(labels)
    
    return loss, output


In [ ]:
#| export
@patch
def evaluation_loop(
        self:XCLearner,
        dataloader: DataLoader,
        description: str,
        prediction_loss_only: Optional[bool] = None,
        ignore_keys: Optional[List[str]] = None,
        metric_key_prefix: str = "eval",
    ) -> XCEvalLoopOutput:
    
    prediction_loss_only = prediction_loss_only if prediction_loss_only is not None else args.prediction_loss_only

    model = self._wrap_model(self.model, training=False, dataloader=dataloader)

    if len(self.accelerator._models) == 0 and model is self.model:
        model = self.accelerator.prepare(model) if self.is_deepspeed_enabled else self.accelerator.prepare_model(model, evaluation_mode=True)
        if self.is_fsdp_enabled: self.model = model
        if model is not self.model: self.model_wrapped = model
        if self.is_deepspeed_enabled: self.deepspeed = self.model_wrapped

    batch_size = self.args.eval_batch_size
    model.eval()
    self.callback_handler.eval_dataloader = dataloader
    eval_dataset = getattr(dataloader, "dataset", None)
    
    if args.past_index >= 0: self._past = None

    losses_host, all_losses = None, None
    host_output, all_output = {}, {}
    
    observed_num_examples = 0
    for step, inputs in enumerate(dataloader):
        observed_batch_size = find_batch_size(inputs)
        if observed_batch_size is not None:
            observed_num_examples += observed_batch_size
            if batch_size is None: batch_size = observed_batch_size
                
        loss, output = self.prediction_step(model, inputs, prediction_loss_only, ignore_keys=ignore_keys)
        
        if loss is not None:
            losses = self.gather_function((loss.repeat(batch_size)))
            losses_host = losses if losses_host is None else nested_concat(losses_host, losses, padding_index=-100)
        for k in output: host_output[k] = self._gather_host_output(output[k], host_output.get(k, None))
            
        self.control = self.callback_handler.on_prediction_step(args, self.state, self.control)
        
        if args.eval_accumulation_steps is not None and (step + 1) % args.eval_accumulation_steps == 0:
            if losses_host is not None: all_losses = losses_host if all_losses is None else nested_concat(all_losses, losses, padding_index=-100)
            for k in host_output: all_output[k], host_output[k] = self._gather_all_output(host_output[k], all_output.get(k, None)), None
    
    self.gather_function = self.accelerator.gather_for_metrics
    if args.past_index and hasattr(self, "_past"): delattr(self, "_past")

    if losses_host is not None: all_losses = losses_host if all_losses is None else nested_concat(all_losses, losses, padding_index=-100)
    for k in host_output: all_output[k], host_output[k] = self._gather_all_output(host_output[k], all_output.get(k, None)), None
        
    if has_length(eval_dataset): num_samples = len(eval_dataset)
    elif isinstance(eval_dataset, IterableDatasetShard) and getattr(eval_dataset, "num_examples", 0) > 0:
        num_samples = eval_dataset.num_examples
    else:
        if has_length(dataloader): num_samples = self.num_examples(dataloader)
        else: num_samples = observed_num_examples
    if num_samples == 0 and observed_num_examples > 0: num_samples = observed_num_examples

    if (self.compute_metrics is not None and 
        'targ_idx' in all_output and all_output['targ_idx'] is not None and 
        'pred_idx' in all_output and all_output['pred_idx'] is not None): 
        metrics = self.compute_metrics(**all_output)
    else: metrics = {}
        
    metrics = denumpify_detensorize(metrics)

    if all_losses is not None: metrics[f"{metric_key_prefix}_loss"] = all_losses.mean().item()
    if hasattr(self, "jit_compilation_time"): metrics[f"{metric_key_prefix}_jit_compilation_time"] = self.jit_compilation_time
        
    for key in list(metrics.keys()):
        if not key.startswith(f"{metric_key_prefix}_"): metrics[f"{metric_key_prefix}_{key}"] = metrics.pop(key)
    
    return XCEvalLoopOutput(pred_idx=all_output['pred_idx'], pred_ptr=all_output['pred_ptr'], pred_score=all_output['pred_score'],
                            targ_idx=all_output['targ_idx'], targ_ptr=all_output['targ_ptr'], metrics=metrics, num_samples=num_samples)
    

### Example

In [ ]:
os.environ['WANDB_MODE'] = 'disabled'

In [ ]:
block = Test.from_cfg('data', valid_pct=0.001)

In [ ]:
args = XCLearningArguments(
    output_dir='/home/scai/phd/aiz218323/scratch/garbage/T1/',
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    eval_steps=10,
    evaluation_strategy='steps',
    label_names=['lbl2data_idx']
)

In [ ]:
model = BT0002.from_pretrained('bert-base-cased', tn_targ=10_000, ig_tok=0)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of BT0002 were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['loss_fn.o']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trie = XCTrie.from_block(block)

  0%|          | 0/312330 [00:00<?, ?it/s]

In [ ]:
metric = PrecRecl(block.n_lbl, block.valid.data_lbl_filterer, prop=block.train.dset.data.data_lbl, pk=5, rk=5, rep_pk=[1, 3, 5], rep_rk=[5])

In [ ]:
learn = XCLearner(
    model=model, 
    args=args,
    trie=trie,
    data_collator=block.collator, 
    train_dataset=block.train.dset, 
    eval_dataset=block.valid.dset,
    compute_metrics=metric,
)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
learn.train()

/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,P@1,P@3,P@5,N@1,N@3,N@5,Psp@1,Psp@3,Psp@5,Psn@1,Psn@3,Psn@5,R@5
10,No log,4.048509,0.023392,0.019006,0.016082,0.023392,0.029194,0.036365,0.009234,0.012774,0.018536,0.009234,0.011317,0.015473,0.052109
20,No log,4.152409,0.046784,0.031189,0.025439,0.046784,0.049224,0.057588,0.031262,0.037875,0.047098,0.031262,0.034214,0.040095,0.072299
30,No log,3.835953,0.040936,0.030702,0.022515,0.040936,0.048425,0.052824,0.020158,0.032948,0.037704,0.020158,0.028077,0.032004,0.064488


/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/scipy/sparse/_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/scipy/sparse/_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


KeyboardInterrupt: 